# Work space

In [ ]:
!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
time: 481 µs (started: 2023-06-06 07:25:51 +00:00)


In [ ]:
# Import nessary libraries
import numpy as np
import pandas as pd
# import country_converter as coco

from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings('ignore')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/16CLBs4KTeHKahxECZFQFwRxk3_khT2VI/data_mining/final_project/Code/source_code
dataset/                      recommendation_thien_2.ipynb  results/
PhoBERT/                      recommendation_thien_3.ipynb  Thien.ipynb
recommendation_kiet.ipynb     recommendation_thien.ipynb
recommendation_thien_1.ipynb  recommendation_v1.ipynb
time: 6.47 s (started: 2023-06-06 07:25:51 +00:00)


# Installation

In [ ]:
# !git clone https://github.com/VinAIResearch/PhoBERT
%cd PhoBERT/
# !git clone --single-branch --branch fast_tokenizers_BARTpho_PhoBERT_BERTweet https://github.com/datquocnguyen/transformers.git
%cd transformers

/content/drive/.shortcut-targets-by-id/16CLBs4KTeHKahxECZFQFwRxk3_khT2VI/data_mining/final_project/Code/source_code/PhoBERT
/content/drive/.shortcut-targets-by-id/16CLBs4KTeHKahxECZFQFwRxk3_khT2VI/data_mining/final_project/Code/source_code/PhoBERT/transformers
time: 7.8 ms (started: 2023-06-06 07:25:58 +00:00)


In [ ]:
!pip3 install .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/drive/.shortcut-targets-by-id/16CLBs4KTeHKahxECZFQFwRxk3_khT2VI/data_mining/final_project/Code/source_code/PhoBERT/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.28.0.dev0-py3-none-any.whl size=7134681 sha256=eb96dfcc03319b10b642de5965c68ac8cbe289751e65cb3812dfe759e343929f
  Stored in directory: /tmp/pip-ephem-wheel-cache-42_tuy4c/wheels/de/4d/7e/b2c81067b8a1b70134881385aa31827ccefd771de5f7138cf6
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.28.0.dev0
    Uninstalling transformers-4.28.0.dev0:
      Successfully uninstalled transformers-4.28.0.dev0
time: 1min 15s (started: 2023-06-06 07:25:58 +00:00)


In [ ]:
!pip3 install tokenizers
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
time: 19.6 s (started: 2023-06-06 07:27:13 +00:00)


# Utils functions

In [ ]:
def add_condition_description(dataset):
    # Additional conditions for better similarity among clusters involving subjects 
    conditions = {
        'nganh_BB': 'Môn học bắt buộc',
        'nganh_BMAV': 'Môn học ngoại ngữ',
        'nganh_CNPM': 'Môn học thuộc khoa Công Nghệ Phần Mềm',
        'nganh_HTTT': 'Môn học thuộc khoa Hệ Thống Thông Tin',
        'nganh_KHMT': 'Môn học thuộc khoa Khoa Học Máy Tính',
        'nganh_KTMT': 'Môn học thuộc khoa Kĩ Thuật Máy Tính',
        'nganh_KTTT': 'Môn học thuộc khoa Kĩ Thuật Thông Tin',
        'nganh_MMT&TT': 'Môn học thuộc khoa Mạng Máy Tính và Truyền Thông'
    }
    for condition, value in conditions.items():
        mask = dataset[condition] == 1
        dataset.loc[mask, 'monhoc_encode'] = dataset['tenmh'].astype(str) + ': ' + dataset['mota'].astype(str) + ' ' + value
    return dataset
    
def filter(dataset, name_col: str, name_value: list):
    data = dataset[dataset[name_col].isin(name_value)]
    return data

time: 6.35 ms (started: 2023-06-06 07:27:33 +00:00)


# Load datasets

In [ ]:
# Load dataset
dataset_original = pd.read_csv('../data/datasets.csv')
print(dataset_original.columns)
dataset_original_usage = dataset_original.copy()
dataset_original = dataset_original[[        \
    'mssv', 'gioitinh', 
    'CNPM', 'HTTT', 'KHMT', 'KTMT', 'KTTT', 'MMT&TT',   \
    'CLC', 'CNTN', 'CQUI', 'CTTT', 'KSTN',  \
    'mamh', 'tenmh' ,'mota',    \
    'nganh_BB', 'nganh_BMAV', 'nganh_CNPM', 'nganh_HTTT', 'nganh_KHMT', 'nganh_KTMT', 'nganh_KTTT', 'nganh_MMT&TT',     \
    'diem_hp',      \
    'trangthai',        \
]]
dataset_original = add_condition_description(dataset_original)
dataset_original = dataset_original.drop_duplicates(subset=['mssv', 'mamh'], keep='last', inplace=False)
dataset = dataset_original.drop(dataset_original[(dataset_original['nganh_BB'] == 1) | (dataset_original['nganh_BMAV'] == 1)].index)


# Create 3 datasets
scores = dataset[['mssv', 'mamh', 'diem_hp']].copy().drop_duplicates()
subjects = dataset[['mamh', 'tenmh', 'monhoc_encode', \
                    'nganh_BB', 'nganh_BMAV', 'nganh_CNPM', 'nganh_HTTT', 'nganh_KHMT', 'nganh_KTMT', 'nganh_KTTT', 'nganh_MMT&TT']].copy().drop_duplicates()
students = dataset[['mssv', 'gioitinh', \
                    'CNPM', 'HTTT', 'KHMT', 'KTMT', 'KTTT', 'MMT&TT', \
                    'CLC', 'CNTN', 'CQUI', 'CTTT', 'KSTN']].copy().drop_duplicates()

Index(['Unnamed: 0', 'mssv', 'gioitinh', 'CNPM', 'HTTT', 'KHMT', 'KTMT',
       'KTTT', 'MMT&TT', 'CLC', 'CNTN', 'CQUI', 'CTTT', 'KSTN', 'nam_th',
       'dien_tt', 'diem_tt', 'mamh', 'tenmh', 'mota', 'nganh_BB', 'nganh_BMAV',
       'nganh_CNPM', 'nganh_HTTT', 'nganh_KHMT', 'nganh_KTMT', 'nganh_KTTT',
       'nganh_MMT&TT', 'sotc', 'hocky', 'namhoc', 'diem_hp', 'trangthai',
       'dtbhk', 'sotchk', 'drl', 'ghichu', 'dtb_toankhoa', 'dtb_tichluy',
       'xeploai', 'dunghan'],
      dtype='object')
time: 4.24 s (started: 2023-06-06 07:27:33 +00:00)


In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer

phobert = AutoModel.from_pretrained("vinai/phobert-base-v2")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

Some weights of the model checkpoint at vinai/phobert-base-v2 were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


time: 16 s (started: 2023-06-06 07:27:37 +00:00)


### Get attention matrix

In [ ]:
def get_attention_matrix(
        dataset=None,
        dataset_original=None,
        phobert=None,
        tokenizer=None):
    # INPUT TEXT MUST BE ALREADY WORD-SEGMENTED!
    monhoc = dict(zip(dataset['mamh'], dataset['monhoc_encode']))
    lst_keys = list(monhoc.keys())
    monhoc['PH001'] = 'Nhập môn điện tử: Đây là môn học ở giai đoan kiến thức đai cương. Môn học này trình bày các khái niệm và ̣phương pháp cơ bản về điện tử. Giới thiệu về nguyên lý hoạt động của của các linh kiện điện tử cơ bản (điện trở, tụ điện, nguồn điện, transistor,….). Ứng dụng các linh kiện điện tử này vào các mạch điện thực tế. Môn học thuộc khoa Kĩ Thuật Máy Tính'
    # monhoc[lst_keys[19]] = 'Xác suất thống kê: Môn học này trình bày các khái niệm và phương pháp về: Lý thuyết xác suất (Không gian xác suất; Biến ngẫu nhiên; Hàm đặc trưng; Dãy các biến ngẫu nhiên; Các quy luật phân phối xác suất; Các định lý giới hạn phân phối xác suất) và Thống kê (Mẫu ngẫu nhiên; Ước lượng điểm và ước lượng khoảng; Kiểm định các giả thiết thống kê; Phân tích tương quan và hồi quy; Một số vấn đề về quá trình ngẫu nhiên). Ngoài ra, môn học này còn giới thiệu về cách thức nhận diện, phân tích và xử lý một vấn đề thực tế; xử lý các số liệu thống kê; để từ đó giúp cho người dùng đưa ra các suy luận phù hợp (nhằm hỗ trợ cho quá trình ra quyết định). Môn học bắt buộc'
    monhoc['SE102'] = 'Nhập môn phát triển game: Kỹ thuật lập trình DirectX để xây dựng các game 2D đơn giản như Tetris, Battle City, Mario, Contras... Nội dung bao gồm: Giới thiệu ngành game, kỹ thuật lập trình Windows dùng C++ và Windows SDK, kỹ thuật làm chuyển động và lập trình DirectX cơ bản, kỹ thuật làm việc với Sprite và xử lý thiết bị nhập, thảo luận về các kỹ thuật hỗ trợ như phép biến đổi, lập trình DirectSound, hiển thị chữ ..., bàn luận về Game Engine và cách xây dựng một game engine đơn giản. Môn học thuộc khoa Công Nghệ Phần Mềm'
    monhoc['SE101'] = 'Phương pháp mô hình hóa: Trình bày các kiến trúc, nền tảng về các phương pháp mô hình hóa thông tin, tri thức, biểu diễn vấn đề và lời giải, mô hình hóa hệ thống. Các phương pháp mô hình hóa và biểu diễn vấn đề như mô hình hóa và biểu diễn dữ liệu, mô hình hóa và biểu diễn quan hệ, mô hình hóa và biểu diễn tiến trình, mô hình hóa và biểu diễn tri thức (SDLC, JSD, SSM, OOA)... làm quen với các công cụ dùng để biểu diễn mô hình như công cụ CASE (upper và lower), các ngôn ngữ mô phỏng mô hình hóa như ngôn ngữ UML, VRML, ... nhằm hiện thực hóa một hệ thống. Học phần bao gồm dẫn nhập và giới thiệu những khái niệm về các mô hình đặc trung hiện nay; giới thiệu về phương pháp luận dùng cho mô hình hóa và giới thiệu cụ thể về các mô hình biểu diễn thông tin, dữ liệu, thời gian thực. Môn học thuộc khoa Công Nghệ Phần Mềm'
    monhoc['NT118'] = 'Phát triển ứng dụng trên thiết bị di động: Thiết kế giao diện cho các thiết bị di động trên Google Android. Xây dựng ứng dụng Native app lẫn cross platform app. Trong ứng dụng native app, sử dụng Java để thể hiện chương trình trên Android; Trong ứng dụng Native app, sinh viên sử dụng HTML, CSS, JavaScript để tạo ra một ứng dụng chuyển tiếp, liên lạc và swipe, hình ảnh động. Tích hợp các dịch vụ web hiện có từ Google và Amazon. Các nội dung bao gồm: Giới thiệu về tính toán di động khắp mọi nơi, tính toán cảm ngữ cảnh, giới thiệu hệ điều hành Android và các phương pháp lập trình trên Android. Các phương pháp lập trình nâng cao: đa luồng, đa hành vi, kết nối SQLite, Web Services; Khái niệm cross platform, thiết kế web di động, ứng dụng cho Điện thoại di động. Đánh dấu cho điện thoại di động. Web Apps di động và tính năng. Giới thiệu PhoneGap. Bản địa hóa ứng dụng. Khoa Mạng Máy Tính và Truyền Thông'
    monhoc['NT205'] = 'Tấn công mạng: Lý thuyết về những lỗ hổng bảo mật phổ biến tồn tại trong hệ thống mạng, hệ điều hành, ứng dụng; Các phương pháp tấn công dựa vào các lỗ hổng đã phát hiện; Các bước thực hiện tấn công chiếm quyền điều khiển hệ thống, thay đổi dữ liệu hay từ chối dịch vụ…; Xây dựng hệ thống phòng thủ ngăn chặn các cuộc tấn công. Đối với hệ Cử nhân tài năng: Trình bày chuyên sâu hơn về các giao thức mạng và việc tận dụng các lỗ hổng trong giao thức để tấn công; cách thức tấn công trên webserver cấu hình mạnh; các phương pháp tấn công ứng dụng web; cách thức tấn công và phòng chống lại các cuộc tấn công mạng trong tương lai; Bổ sung các bài tập nâng cao về việc sử dụng các công cụ crack password phức tạp và leo thang đặc quyền, xoá dấu vết, tấn công DDoS, cách thức điều khiển các zombie và xây dựng các mạng BotNet. Môn học thuộc khoa Mạng Máy Tính và Truyền Thông'
    monhoc['SE334'] = 'Các phương pháp lập trình: Học phần này trình bày các kiến trúc, nền tảng về các phương pháp, kỹ thuật lập trình thường dùng khi thiết kế và xây dựng một chương trình máy tính. Ngôn ngữ C++, Java, các thư viện hỗ trợ trong lập trình song song. Học phần gồm: Giới thiệu các kỹ thuật và các nguyên lý cơ bản của lập trình; Giới thiệu cụ thể về các phương pháp và kỹ thuật lập trình như đệ qui, tối ưu mã chương trình, phương pháp lập trình cấu trúc, lập trình hướng đối tượng, lập trình đa nhiệm, song song; Giới thiệu kỹ thuật thiết kế kiến trúc và giao diện chương trình. Môn học thuộc khoa Công Nghệ Phần Mềm'
    monhoc['NT505'] = 'Khóa luận tốt nghiệp: Một công trình nghiên cứu khoa học dành cho sinh viên. Trong khóa luận, sinh viên nêu rõ những vấn đề do sinh viên thực hiện được dưới sự hướng dẫn của giảng viên như: ứng dụng, quy trình hoạt động, hệ thống triển khai. Ngoài ra khóa luận cần có những đánh giá, phương hướng phát triển tiếp theo của đề tài. Trong khóa luận nêu rõ kết quả thực hiện của sinh viên, đây là thành phần quan trọng nhất của khóa luận. Đề tài khóa luận tốt nghiệp là một đề tài được nghiên cứu và triển khai chuyên sâu gắn với yêu cầu thực tế cho thấy khả năng nghiên cứu và làm việc độc lập của sinh viên. Trong khóa luận tốt nghiệp, cần xác định rõ những vấn đề do sinh viên thực hiện được dưới sự hướng dẫn của giảng viên như: ứng dụng, quy trình hoạt động, hệ thống triển khai, tính mới của nghiên cứu. Ngoài ra khóa luận cần có những đánh giá, phương hướng phát triển tiếp theo của đề tài. Trong khóa luận cần nêu rõ kết quả nghiên cứu của sinh viên. Môn học thuộc khoa Mạng Máy Tính và Truyền Thông'
    monhoc['NT211'] = 'An ninh nhân sự, định danh và chứng thực: Khái niệm căn bản về định danh, xác thực và ứng dụng của chúng trong quản lý truy cập. Môn học trang bị cho sinh viên ngành an ninh thông tin: Khái niệm nền tảng về an ninh liên quan tới con người; Kiến thức về định danh cùng các công nghệ định danh hiện đại; Kiến thức về xác thực và những công nghệ liên quan đến xác thực; Ứng dụng định danh và xác thực trong hệ thống CNTT. Đối với hệ Cử nhân tài năng: Trình bày chuyên sâu hơn các nội dung Sinh trắc và các phương pháp chính; Quản lý tài khoản với Token; Quản lý tài khoản liên hợp; Tấn công thẻ thông minh; Bổ sung các bài tập nâng cao ở các nội dung trình bày chuyên sâu trên và các nội dung Bẻ mật khẩu phức tạp; Thiết kế, xây dựng và triển khai hệ thống cấp chứng chỉ số ở quy mô lớn. Môn học thuộc khoa Mạng Máy Tính và Truyền Thông'
    monhoc['SE220'] = 'Thiết kế Game: Kiến thức, kỹ năng lĩnh vực thiết kế game. Lý thuyết về diễn biến tâm lý người chơi, bản chất của game, tại sao game hấp dẫn. Kỹ thuật thiết kế game, lịch sử trong thiết kế game. Tập trung vào thiết kế giao diện game như cách xây dựng menu, bố trí các thành phần giao diện, biểu tượng, thiết kế HUD. Bàn về thiết kế cảnh chơi như cách đặt thử thách, xây dựng bối cảnh, tạo hồn cho cảnh chơi... Môn học thuộc khoa Công Nghệ Phần Mềm'
    monhoc['SE320'] = 'Lập trình đồ họa 3 chiều với Direct3D: Lập trình ứng dụng đồ họa 3 chiều và hướng dẫn sử dụng bộ thư viện đồ họa DirectX để xây dựng ứng dụng. Trình bày về cơ sở toán học ứng dụng trong đồ họa 3 chiều và quy trình dựng hình 3 chiều, trình bày về Direct3D bao gồm các vấn đề đi từ cơ bản đến nâng cao, ứng dụng các kiến thức đã học vào xây dựng trò chơi Tetris 3D. Kết thúc khóa học, sinh viên sẽ có khả năng tự thiết kế và lập trình ứng dụng đồ họa 3 chiều đơn giản trên môi trường Windows. Môn học thuộc khoa Công Nghệ Phần Mềm'
    monhoc['NT534'] = 'An toàn mạng máy tính nâng cao: Cách phòng chống tấn công từ chối dịch vụ, các hoạt động ngầm trên Internet, bàn luận về các giải pháp kĩ thuật trong việc ngăn chặn cũng như đối phó với ngăn chặn trong việc quản lý truy cập trên Internet. Ngoài ra, môn này cũng đề cập các nguy cơ từ các loại mã độc tinh vi đối với an toàn mạng. Đối với hệ tài năng: Môn an toàn mạng đề cập các chủ đề căn bản của an toàn mạng. Môn này đề cập đến các vấn đề chuyên sâu hơn ví dụ như là làm thế nào để phòng chống tấn công từ chối dịch vụ, các hoạt động ngầm trên Internet, bàn luận về các giải pháp kĩ thuật trong việc ngăn chặn cũng như đối phó với ngăn chặn trong việc quản lý truy cập trên Internet. Ngoài ra, môn này cũng đề cập các nguy cơ từ các loại mã độc tinh vi đối với an toàn mạng. Cuối cùng, các kỹ thuật client side, server-side honeypot cũng được giới thiệu để nghiên cứu, thu thập mã độc. Môn học thuộc khoa Mạng Máy Tính và Truyền Thông'
    monhoc['IE307'] = 'Công nghệ lập trình đa nền tảng cho ứng dụng di động: Môn học trình bày nguyên lý cơ bản của các Framework về lập trình di động đa nền tảng (React Native, PhoneGap, Xamarin...) và đặc biệt là Xamarin Framework. Cung cấp các Controls cơ bản của Xamarin, và áp dụng để xây dựng ứng dụng đa nền tảng: Label, Entry, Button, Image, Switch, ListView, DatePicker, TimePicker. Bên cạnh đó, môn học còn cung cấp thêm các vấn đề nâng cao của Xamarin, để tiếp tục tự nghiên cứu sử dụng về sau của Camera, Notification, Google Map APIs, Grial, RESTful API, Syncfusion... Môn học trang bị kỹ năng làm việc nhóm theo môi trường doanh nghiệp, đọc hiểu yêu cầu của khách hàng về ứng dụng di động, Phân tích & Thiết kế các ứng dụng di động để xây dựng một ứng dụng di động đa nền tảng cơ bản chạy trên iOS, Android & Windows Phone theo yêu cầu. Môn học thuộc khoa Kĩ Thuật Thông Tin'
    lst_input_ids = [torch.tensor([tokenizer.encode(monhoc[key])]) for key in lst_keys]

    with torch.no_grad():
        features = {}
        for index, input_ids in enumerate(lst_input_ids):
            features[lst_keys[index]] = phobert(input_ids).pooler_output

    lst_supervision_extra_dic = {
        'nganh_CNPM': 'Cung cấp sự hiểu biết các đặc trưng chính của phần mềm, khái niệm chu trình phần mềm, các hoạt động kỹ thuật, cung cấp kiến thức thực nghiệm về chọn lựa kỹ thuật, công cụ, mô hình chu trình dự án, các kiến thức độ quan trọng đảm bảo chất lượng (quality assurance), quản lý dự án trong phát triển phần mềm',
        'nganh_HTTT': 'Nghiên cứu các hệ thống thông tin quản trị doanh nghiệp, ngân hàng như ERP, Supply Chain Management; Nghiên cứu các ứng dụng xây dựng hệ thống thông tin phục vụ Thương Mại Điện Tử; Phát triển các nghiên cứu nhằm tăng cường khai thác tri thức từ CSDL, quản trị các kho dữ liệu lớn, tìm kiếm thông tin trên web, tìm kiếm ngữ nghĩa, mạng xã hội; Phát triển các nghiên cứu liên ngành giữa tin học và các ngành khoa học khác như: xử lý ngôn ngữ tự nhiên, sinh học, hoá học, môi trường, ...',
        'nganh_KHMT': 'Đào tạo bài bản về Trí tuệ nhân tạo (Artificial Intelligence - AI) đáp ứng nhu cầu về nghiên cứu, xây dựng và phát triển các sản phẩm, giải pháp thông minh phục vụ cho cuộc sống. Chương trình đào tạo của Khoa cung cấp cho sinh viên nhiều lựa chọn theo các định hướng nghề nghiệp như Trí tuệ Nhân tạo (AI), Thị giác Máy tính (Computer Vision), Xử lý Ngôn ngữ Tự nhiên (Natural Language Processing)…. Với các kiến thức nền tảng sinh viên hoàn toàn có thể tham gia nghiên cứu và phát triển các ứng dụng thông minh như: hệ thống nhận diện khuôn mặt (Face Recognition System), hệ thống Chatbot, hệ thống tìm kiếm – truy vấn thông tin (Retrieval System) ...',
        'nganh_KTMT': 'Lập trình các phần mềm nhúng trên các thiết bị di động (Smartphone, Tablet, iphone, ipad, ...), các vi xử lý-vi điều khiển trong các hệ thống công nghiệp, xe ô tô, điện gia dụng, ngôi nhà thông minh,… (Chuyên ngành hệ thống nhúng và IoT); thiết kế mạch điện - điện tử, mạch điều khiển trong công nghiệp, vi mạch, chip,... (Chuyên ngành thiết kế vi mạch)',
        'nganh_KTTT': 'Thiết kế, xây dựng và quản lý các dự án nghiên cứu và ứng dụng CNTT, chủ yếu trong lĩnh vực dữ liệu không gian-thời gian (địa lý, tài nguyên, môi trường, viễn thám. . .). Tập trung vào những ứng dụng về GIS trên thiết bị di động và trao đổi dữ liệu với máy chủ; Vận hành, quản lý, giám sát; phân tích và phát triển các ứng dụng CNTT tại các doanh nghiệp; Khai thác dữ liệu và thông tin ứng dụng cho các doanh nghiệp trong vấn đề phân tích định lượng; Xây dựng, phát triển các ứng dụng về lãnh vực truyền thông xã hội và công nghệ Web',
        'nganh_MMT&TT': 'Quản trị mạng và hệ thống tại các ngân hàng, các trung tâm dữ liệu, các nhà cung cấp dịch vụ Internet (ISP); Thiết kế mạng chuyên nghiệp: xây dựng các mạng máy tính an toàn, hiệu quả cho các đơn vị có yêu cầu; Phát triển phần mềm mạng; Phát triển phần mềm mạng; Xây dựng và phát triển các ứng dụng truyền thông: VoIP, hội nghị truyền hình',
    }
    lst_supervision_keys = list(lst_supervision_extra_dic.keys())
    lst_supervision_token = [torch.tensor([tokenizer.encode(lst_supervision_extra_dic[key])]) for key in ['nganh_CNPM', 'nganh_HTTT', 'nganh_KHMT', 'nganh_KTMT', 'nganh_KTTT', 'nganh_MMT&TT']]
    with torch.no_grad():
        features_supervision = {}
        for index, input_ids in enumerate(lst_supervision_token):
            features_supervision[lst_supervision_keys[index]] = phobert(input_ids).pooler_output

    data_temp = dataset_original[['mamh', 'nganh_BB', 'nganh_BMAV', 'nganh_CNPM', 'nganh_HTTT', 'nganh_KHMT', 'nganh_KTMT', 'nganh_KTTT', 'nganh_MMT&TT']]

    conditions = [
        dataset_original['nganh_BB'] == 1,
        dataset_original['nganh_BMAV'] == 1,
        dataset_original['nganh_CNPM'] == 1,
        dataset_original['nganh_HTTT'] == 1,
        dataset_original['nganh_KHMT'] == 1,
        dataset_original['nganh_KTMT'] == 1,
        dataset_original['nganh_KTTT'] == 1,
        dataset_original['nganh_MMT&TT'] == 1   
    ]
    values = ['nganh_BB', 'nganh_BMAV', 'nganh_CNPM', 'nganh_HTTT', 'nganh_KHMT', 'nganh_KTMT', 'nganh_KTTT', 'nganh_MMT&TT']
    data_temp['nganh'] = np.select(conditions, values, default=0)

    data_temp = data_temp.drop_duplicates()

    for index, key in enumerate(features.keys()):
        val = data_temp.loc[data_temp['mamh'] == key, 'nganh'].values[0]
        if val in ['nganh_BB', 'nganh_BMAV']:
            continue
        else:
            features[key] = torch.cat((features[key], features_supervision[val]), 1)
    data = [{
            'mh1': i, 
            'mh2': j, 
            'similarity': cosine_similarity(features[i], features[j]).item()
        } \
        for i in features.keys() \
        for j in features.keys()]
    attention_monhoc = pd.DataFrame(data)
    attention_matrix = attention_monhoc.pivot_table(index=["mh1"], columns=["mh2"], values="similarity")
    # attention_matrix.to_csv('../../dataset/attention_matrix.csv')
    return attention_matrix

time: 10.5 ms (started: 2023-06-06 07:27:53 +00:00)


### Get related subjects

In [ ]:
# Phase 1
def get_related_subjects(
        mamh_query=None, 
        attention_matrix=None, 
        k_threshold=20):
    
    si = attention_matrix[mamh_query]
    rcm_df = pd.DataFrame(attention_matrix.corrwith(si).sort_values(ascending=False)).reset_index(drop=False)
    rcm_df = rcm_df.rename(columns={rcm_df.columns[0]: 'mamh'})
    related_subjects = rcm_df.head(k_threshold)['mamh'].tolist()
    return related_subjects

time: 1.12 ms (started: 2023-06-06 07:27:53 +00:00)


### Get related students

In [ ]:
# Phase 2
def get_related_students(
        dataset_original=None, 
        mamh_query=None, 
        mssv_query=None,
        t_threshold=8):
    
    df_students = dataset_original[[
        'mssv', 'gioitinh', 
        'CNPM', 'HTTT', 'KHMT', 'KTMT', 'KTTT', 'MMT&TT',   \
        'CLC', 'CNTN', 'CQUI', 'CTTT', 'KSTN',  \
        'mamh',     \
        'diem_hp',      \
    ]]
    df_students = df_students[df_students['mamh'].isin(['IT001', 'IT002' ,'IT003', 'IT004', 'IT005', 'IT006', 'IT007'] + [mamh_query])]
    one_hot_encoded = pd.pivot_table(data=df_students, index=df_students.index, columns='mamh', values='diem_hp', fill_value=-10.0)

    # Concatenate the one-hot encoded columns with the original DataFrame
    df_students_encoded = pd.concat([df_students, one_hot_encoded], axis=1)
    df_students_merged = df_students_encoded.groupby('mssv').max().reset_index()
    df_students_merged = df_students_merged.drop(columns=['mamh', 'diem_hp'])
    
    similarity_matrix = cosine_similarity(df_students_merged.iloc[:, 1:])
    similarity_df = pd.DataFrame(similarity_matrix, index=df_students_merged['mssv'], columns=df_students_merged['mssv'])
    float_list = np.arange(0.993, 0.998 + 0.0001, 0.0001).tolist()
    for thr in float_list:
        list_related_students = similarity_df[(similarity_df[mssv_query] > thr) & (similarity_df.index != mssv_query)].index
        if len(list_related_students) != 0 and len(list_related_students) <= t_threshold:
            break
    return list_related_students

time: 1.83 ms (started: 2023-06-06 07:27:53 +00:00)


### Get score single

In [ ]:
# Phase calculate score
def get_score_single(
        dataset_original,
        list_related_subjects=None,
        list_related_students=None,
        mamh_query=None,
        mssv_query=None):
    
    if len(list_related_students) == 0:
        return [], 0.0
    for index, student in enumerate(list_related_students):
        history_subjects = list(dataset_original[dataset_original['mssv'].isin([student])]['mamh'].unique())
        if index == 0:
            intersection = list(set(history_subjects) & set(list_related_subjects))
        else:
            intersection = list(set(history_subjects) & set(intersection))

    intersection.remove(mamh_query)
    if len(intersection) == 0:
        return [], 0.0
    sv_subjects = list(dataset_original[dataset_original['mssv'].isin([mssv_query])]['mamh'].unique())
    result = list(set(sv_subjects) & set(intersection))
    score_result = len(result) / len(intersection)
    return (intersection, score_result)

time: 1.47 ms (started: 2023-06-06 07:27:53 +00:00)


# Inference 1 data point

In [ ]:
# ten_mh = 'CS112'
# mssv = 'EAA0B693XPvAibaEXe99j2P15eeB04XwhZ0tzlI4'
# attention_matrix = pd.read_csv('../../dataset/attention_matrix.csv')
# ### Retrained entire to get Attention matrix, this progress take about 2, 3 minutes
# attention_matrix = get_attention_matrix(
#     dataset=dataset,
#     dataset_original=dataset_original,
#     phobert=phobert,
#     tokenizer=tokenizer
# )

time: 404 µs (started: 2023-06-06 07:27:53 +00:00)


In [ ]:
# from sklearn.metrics.pairwise import cosine_similarity
# list_related_subjects = get_related_subjects(
#     mamh_query=ten_mh, 
#     attention_matrix=attention_matrix,
#     k_threshold=20)

# list_related_students = get_related_students(
#     dataset_original=dataset_original, 
#     mamh_query=ten_mh, 
#     mssv_query=mssv,
#     t_threshold=0.995)

time: 410 µs (started: 2023-06-06 07:27:53 +00:00)


In [ ]:
# result = get_score_single(
#     dataset_original=dataset_original,
#     list_related_subjects=list_related_subjects,
#     list_related_students=list_related_students,
#     mamh_query=ten_mh,
#     mssv_query=mssv)

# result

time: 439 µs (started: 2023-06-06 07:27:53 +00:00)


# Training models

In [ ]:
# Training
from tqdm import tqdm
attention_matrix = pd.read_csv('../../dataset/attention_matrix.csv')
i = 15
while 15 <= i and i <= 22:
    j = 3
    while 3 <= j and j <= 10:
        dataset_train = dataset_original.drop(dataset_original[(dataset_original['nganh_BB'] == 1) | (dataset_original['nganh_BMAV'] == 1)].index)
        lst_users = list(dataset_train['mssv'].unique())
        score = 0.0
        lst_count = [0, 0, 0, 0, 0, 0, 0, 0, 0]

        print(f'\nthreshold_get_subjects: {i}, threshold_get_students: {j}')

        for index_user, user in tqdm(enumerate(lst_users)):

            if index_user == 5:
                break

            data_user = filter(dataset_train, 'mssv', [user])
            data_user.drop_duplicates(subset=['mssv', 'mamh'], keep='last', inplace=False)

            lst_user_mamh = list(data_user['mamh'])
            # print(f'\nUser: {index_user+1} with the number of subjects: {len(lst_user_mamh)} -------------')
            result_user_single = 0.0
            for index, user_mamh in enumerate(lst_user_mamh):
                list_related_subjects = get_related_subjects(
                    mamh_query=user_mamh, 
                    attention_matrix=attention_matrix,
                    k_threshold=i)
                
                list_related_students = get_related_students(
                    dataset_original=dataset_original, 
                    mamh_query=user_mamh, 
                    mssv_query=user,
                    t_threshold=j)
                
                list_result, result = get_score_single(
                    dataset_original=dataset_original,
                    list_related_subjects=list_related_subjects,
                    list_related_students=list_related_students,
                    mamh_query=user_mamh,
                    mssv_query=user)
                # print(f'\nUser: {user} ------ Subject: {user_mamh} ------ list: {list_result} ------ Score = {round(result, 2)}')
                if len(list_result) <= 7:
                    lst_count[len(list_result)] += 1
                else:
                    lst_count[-1] += 1

                result_user_single += result
            # print(f'Total loss single for {user}: {result_user_single / len(lst_user_mamh)}')
            # print(f'\n================================================================================\n')
            
            score += (result_user_single / (index + 1))

        print(f'Total score: {score * 100 / (index_user + 1)}')
        print(f'Count: {lst_count}')

        with open('../../example.txt', 'a') as file:
            file.write(f'threshold_get_subjects: {i}, threshold_get_students: {j}, score: {score * 100 / 5}, count: {lst_count}\n')

        j += 1
    i += 1

### Add data

In [ ]:
def input_prediction(
    _mssv = None,
    _gioitinh = None,
    _khoa = None,
    _hedaotao = None,
    _mamh = None,
    _ten_mh = None,
    _diem_hp = None,
):
    new_data_point = {
        'mssv': _mssv,
        'gioitinh': 1 if _gioitinh == 'Nam' else 0,
        # khoa
        'CNPM': 1 if _khoa == 'CNPM' else 0,
        'HTTT': 1 if _khoa == 'HTTT' else 0,
        'KHMT': 1 if _khoa == 'KHMT' else 0,
        'KTMT': 1 if _khoa == 'KTMT' else 0,
        'KTTT': 1 if _khoa == 'KTTT' else 0,
        'MMT&TT': 1 if _khoa == 'MMT&TT' else 0,

        # he dao tao
        'CLC': 1 if _hedaotao == 'CLC' else 0,
        'CNTN': 1 if _hedaotao == 'CNTN' else 0,
        'CQUI': 1 if _hedaotao == 'CQUI' else 0,
        'CTTT': 1 if _hedaotao == 'CTTT' else 0,
        'KSTN': 1 if _hedaotao == 'KSTN' else 0,

        # diem hoc phan
        'diem_hp': _diem_hp,
    }

    # new_data_point
    return new_data_point

time: 1.3 ms (started: 2023-06-06 07:28:38 +00:00)


In [ ]:
dataset_original = dataset_original.append([
    input_prediction(_mssv = '20521494', _gioitinh = 'Nam', _khoa = 'KHMT', _hedaotao = 'CQUI', _mamh = 'IT001', _diem_hp = 8.1,),
    input_prediction(_mssv = '20521494', _gioitinh = 'Nam', _khoa = 'KHMT', _hedaotao = 'CQUI', _mamh = 'IT002', _diem_hp = 8.5,),
    input_prediction(_mssv = '20521494', _gioitinh = 'Nam', _khoa = 'KHMT', _hedaotao = 'CQUI', _mamh = 'IT003', _diem_hp = 9.4,),
    input_prediction(_mssv = '20521494', _gioitinh = 'Nam', _khoa = 'KHMT', _hedaotao = 'CQUI', _mamh = 'IT004', _diem_hp = 7.7,),
    input_prediction(_mssv = '20521494', _gioitinh = 'Nam', _khoa = 'KHMT', _hedaotao = 'CQUI', _mamh = 'IT005', _diem_hp = 8.1,),
    input_prediction(_mssv = '20521494', _gioitinh = 'Nam', _khoa = 'KHMT', _hedaotao = 'CQUI', _mamh = 'IT006', _diem_hp = -10.0,),
    input_prediction(_mssv = '20521494', _gioitinh = 'Nam', _khoa = 'KHMT', _hedaotao = 'CQUI', _mamh = 'IT007', _diem_hp = 9.3,),
], ignore_index=True)

time: 35.5 ms (started: 2023-06-06 07:37:03 +00:00)


In [ ]:
from datetime import datetime

# Get the current time
current_time = datetime.now()

# Print the current time
print(type(current_time))

<class 'datetime.datetime'>


In [ ]:
from datetime import datetime
import pytz

# Set the desired timezone (Vietnam)
timezone = pytz.timezone('Asia/Ho_Chi_Minh')

# Get the current time in Vietnam
current_time = datetime.now(timezone)

# Print the current time
print(current_time)
year = current_time.year
month = current_time.month
day = current_time.day
hour = current_time.hour
minute = current_time.minute
second = current_time.second
microsecond = current_time.microsecond

print(year, month, day, hour, minute, second, microsecond)

2023-06-07 00:19:57.909699+07:00
2023 6 7 0 19 57 909699
